<a href="https://colab.research.google.com/github/Andres8bit/Machine-Learning/blob/main/LeNet_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.datasets import mnist
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from keras.utils import np_utils
%matplotlib inline

In [2]:
def learning_rate_scheduler(epoch):
  if epoch <=2:
    lr = 5e-4
  elif epoch > 2 and epoch <= 5:
    lr = 2e-4
  elif epoch > 5 and epoch <= 9:
    lr = 5e-5
  else:
    lr = 1e-5
  return lr

In [3]:
def normalize_images(items):
  temp = np.zeros(shape=(items.shape))
  mean = np.mean(items,axis=(0,1,2))
  std = np.std(items,axis=(0,1,2))
  temp =(items - mean) /(std +1e-7)
  return temp

In [4]:
# Load and Normalize MINST Training data:
(X_train,Y_train), (X_test,Y_test) = mnist.load_data()
X_train = normalize_images(X_train)
X_test = normalize_images(X_test)

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
# One-Hot Encode Labels:
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test,10)

In [7]:
# Shape data to fit model:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

In [8]:
# Model Architecture for LeNet-5:
model = Sequential()

# 1st Convolutional Layer:
model.add(Conv2D(filters=6,kernel_size=5,strides=1,activation='tanh',input_shape=input_shape,padding='same'))
# Pooling Layer:
model.add(AveragePooling2D(pool_size=2,strides=2,padding='valid'))

# 2nd Convolutional Layer:
model.add(Conv2D(filters=16,kernel_size=5,strides=1,activation='tanh',padding='valid'))
# Pooling Layer:
model.add(AveragePooling2D(pool_size=2,strides=2,padding='valid'))

# 3rd Convolutional Layer:
model.add(Conv2D(filters=120,kernel_size=5,strides=1,activation='tanh',padding='valid'))

# Flatten layers for fully connected layer:
model.add(Flatten())

# Fully Connected Layer:
model.add(Dense(units=84,activation='tanh'))
# SoftMax Layer:
model.add(Dense(units=10,activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 120)         48120     
_________________________________________________________________
flatten (Flatten)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 84)                1

In [17]:
# Learingin Rate Scheduler,CheckPoint, and Callback setup:
lr_rate = LearningRateScheduler(learning_rate_scheduler)
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5',verbose=1,save_best_only=True)
callbacks =[checkpoint,lr_rate]

In [11]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')

In [18]:
hist = model.fit(X_train, Y_train, batch_size=32, epochs=20,
          validation_data=(X_test, Y_test), callbacks=[checkpointer, lr_rate], 
          verbose=2, shuffle=True)


Epoch 1/20
1875/1875 - 5s - loss: 0.0330 - accuracy: 0.9891 - val_loss: 0.0469 - val_accuracy: 0.9853

Epoch 00001: val_loss improved from inf to 0.04689, saving model to model.weights.best.hdf5
Epoch 2/20
1875/1875 - 5s - loss: 0.0224 - accuracy: 0.9929 - val_loss: 0.0337 - val_accuracy: 0.9885

Epoch 00002: val_loss improved from 0.04689 to 0.03370, saving model to model.weights.best.hdf5
Epoch 3/20
1875/1875 - 5s - loss: 0.0200 - accuracy: 0.9938 - val_loss: 0.0351 - val_accuracy: 0.9879

Epoch 00003: val_loss did not improve from 0.03370
Epoch 4/20
1875/1875 - 5s - loss: 0.0092 - accuracy: 0.9977 - val_loss: 0.0297 - val_accuracy: 0.9905

Epoch 00004: val_loss improved from 0.03370 to 0.02969, saving model to model.weights.best.hdf5
Epoch 5/20
1875/1875 - 5s - loss: 0.0078 - accuracy: 0.9980 - val_loss: 0.0309 - val_accuracy: 0.9900

Epoch 00005: val_loss did not improve from 0.02969
Epoch 6/20
1875/1875 - 5s - loss: 0.0066 - accuracy: 0.9985 - val_loss: 0.0332 - val_accuracy: 0.98

In [20]:
model.load_weights('model.weights.best.hdf5')

In [23]:
score = model.evaluate(X_test,Y_test,verbose=0)
accuracy = score[1] *100
print('Test Accuracy: %.4f%%' % accuracy)

Test Accuracy: 99.1200%
